In [6]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from astropy import units as u
import scipy

import setigen as stg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Sometimes it can be necessary to re-run this command for plots to show automatically
%matplotlib inline

In [8]:
subsample_factor = 128

sample_rate = 3e9 // subsample_factor
num_taps = 8
num_branches = 1024 // subsample_factor
print(f'Max "num_chans" is {num_branches // 2}.')

fftlength = 1048576
int_factor = 51
num_blocks = 1

digitizer = stg.voltage.RealQuantizer(target_fwhm=8,
                                      num_bits=8)

filterbank = stg.voltage.PolyphaseFilterbank(num_taps=num_taps, 
                                             num_branches=num_branches)

requantizer = stg.voltage.ComplexQuantizer(target_fwhm=8,
                                           num_bits=8)

antenna = stg.voltage.Antenna(sample_rate=sample_rate, 
                              fch1=6*u.GHz,
                              ascending=True,
                              num_pols=2)

block_size = stg.voltage.get_block_size(num_antennas=1,
                                        tchans_per_block=1,
                                        num_bits=8,
                                        num_pols=2,
                                        num_branches=1024,
                                        num_chans=num_branches//2,
                                        fftlength=fftlength,
                                        int_factor=int_factor)
block_size

Max "num_chans" is 4.


855638016

In [9]:
134217728 / (512 * 2 * 2 * fftlength * int_factor)

0.0012254901960784314

In [10]:
num_branches

8

In [11]:
rvb = stg.voltage.RawVoltageBackend(antenna,
                                    digitizer=digitizer,
                                    filterbank=filterbank,
                                    requantizer=requantizer,
                                    start_chan=0,
                                    num_chans=num_branches//2,
                                    block_size=block_size,
                                    blocks_per_file=128,
                                    num_subblocks=32)
signal_level = stg.voltage.get_level(100, 
                                     rvb,
                                     # num_blocks=10, 
                                     # length_mode='num_blocks',
                                     obs_length=300, 
                                     length_mode='obs_length',
                                     fftlength=fftlength)
signal_level

0.0010864381489468279

In [12]:
stg.voltage.get_unit_drift_rate(rvb, fftlength, int_factor)

0.15306383611560062

In [21]:
rvb.get_num_blocks(300)

0

In [ ]:
def add_signal(frame,
               path,
               t_profile,
               f_profile,
               bp_profile=None,
               bounding_f_range=None,
               integrate_path=False,
               integrate_t_profile=False,
               doppler_smearing=False,
               t_subsamples=10):
    """
    Alternate add_signal method for better doppler smearing functionality.

    """
    if bounding_f_range is None:
        bounding_min, bounding_max = 0, frame.fchans
    else:
        bounding_min = max(frame.get_index(bounding_f_range[0]), 0)
        bounding_max = min(frame.get_index(bounding_f_range[1]), frame.fchans)

    restricted_fs = frame.fs[bounding_min:bounding_max]
    ff, _ = np.meshgrid(restricted_fs, frame.ts)

    # Handle t_profile
    if callable(t_profile):
        # Integrate in time direction to capture temporal variations more
        # accurately
        if integrate_t_profile:
            new_ts = np.linspace(0,
                                 frame.tchans * frame.dt,
                                 frame.tchans * t_subsamples,
                                 endpoint=False)
            y = t_profile(new_ts)
            if not isinstance(y, np.ndarray):
                y = np.repeat(y, frame.tchans * t_subsamples)
            integrated_y = np.mean(np.reshape(y, (frame.tchans,
                                                  t_subsamples)),
                                   axis=1)
            t_profile = integrated_y
        else:
            t_profile = t_profile(frame.ts)
    elif isinstance(t_profile, (list, np.ndarray)):
        t_profile = np.array(t_profile)
        if t_profile.shape != frame.ts.shape:
            raise ValueError('Shape of t_profile array is {0} != {1}.'
                             .format(t_profile.shape, frame.ts.shape))
    elif isinstance(t_profile, (int, float)):
        t_profile = np.full(frame.tchans, t_profile)
    else:
        raise TypeError('t_profile is not a function, array, or float.')
    _, t_profile_tt = np.meshgrid(restricted_fs, t_profile)

    # Handle path. Generate one extra time sample for freq smearing
    # calculations
    tchans_eff = frame.tchans
    if doppler_smearing:
        tchans_eff += 1
    if callable(path):
        # Average using integration to get a better position in frequency
        # direction
        if integrate_path:
            new_ts = np.linspace(0,
                                 tchans_eff * frame.dt,
                                 tchans_eff * t_subsamples,
                                 endpoint=False)
            f = path(new_ts)
            if not isinstance(f, np.ndarray):
                f = np.repeat(f, tchans_eff * t_subsamples)
            integrated_f = np.mean(np.reshape(f, (tchans_eff,
                                                  t_subsamples)),
                                   axis=1)
            path = integrated_f
        else:
            ts = frame.ts
            if doppler_smearing:
                ts = np.append(frame.ts, frame.ts[-1] + frame.dt)
            path = path(ts)
    elif isinstance(path, (list, np.ndarray)):
        path = np.array(path)
        if path.shape != frame.ts.shape:
            raise ValueError(f'Shape of path array is {path.shape} '
                             f'!= {frame.ts.shape}.')
        elif doppler_smearing and len(path) != frame.tchans + 1:
            raise ValueError(f'To Doppler smear power, must provide'
                             f'path array with {frame.tchans + 1} values')
    elif isinstance(path, (int, float)):
        path = np.full(tchans_eff, path)
    else:
        raise TypeError('path is not a function, array, or float.')
    # Ensure that path f_centers are the right length
    _, path_tt = np.meshgrid(restricted_fs, path[:frame.tchans])

    if doppler_smearing:
        dpath = np.diff(path) / t_subsamples
        _, dpath_tt = np.meshgrid(restricted_fs, dpath)

    # Handle bandpass profile
    if bp_profile is None:
        bp_profile = 1
    if callable(bp_profile):
        bp_profile = bp_profile(restricted_fs)
    elif isinstance(bp_profile, (list, np.ndarray)):
        bp_profile = np.array(bp_profile)
        if bp_profile.shape != restricted_fs.shape:
            raise ValueError('Shape of bp_profile array is {0} != {1}.'
                             .format(bp_profile.shape,
                                     restricted_fs.shape))
    elif isinstance(bp_profile, (int, float)):
        bp_profile = np.full(restricted_fs.shape, bp_profile)
    else:
        raise TypeError('bp_profile is not a function, array, or float.')
    bp_profile_ff, _ = np.meshgrid(bp_profile, frame.ts)

    # Create signal, adding multiple copies for Doppler smearing case
    if doppler_smearing:
        signal = np.zeros(ff.shape)
        for _ in range(t_subsamples):
            signal += (t_profile_tt * f_profile(ff, path_tt) 
                       / t_subsamples * bp_profile_ff)
            path_tt += dpath_tt
    else:
        signal = t_profile_tt * f_profile(ff, path_tt) * bp_profile_ff


    frame.data[:, bounding_min:bounding_max] += signal

    signal_frame = np.zeros(frame.shape)
    signal_frame[:, bounding_min:bounding_max] = signal

    return signal_frame